Сегодня мы поговорим о морфологических анализаторах для русского языка. Важно отметить, что (в основном) для каждого естественного языка нужен свой морфоанализатор.

* шаг 1: установим пайморфи и майстем

In [ ]:
# !pip3 install pymystem3
# !pip3 install pymorphy2[fast]

# ! pip3 install gensim

In [1]:
# импортируем нужные части библиотек
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

# сохраняем класс в переменную
mystem = Mystem() 
morph = MorphAnalyzer() 


In [2]:
# сэмпл-текст, на котором все будем пробовать
text = """Зрелый фрукт на вкус очень сладок и обладает приятным сладковатым ароматом. В нём много витаминов и сахаров, но мало кислот."""

## нормализация

давайте приведем текст к нижнему регистру

In [3]:
text = text.lower()
text

'зрелый фрукт на вкус очень сладок и обладает приятным сладковатым ароматом. в нём много витаминов и сахаров, но мало кислот.'

## токенизация

иногда текст нужно предварительно разбить на токены, отдельные элементы. Токенизаторов много, каждый делает это, руководствуясь своими правилами

* токенизация нужна, если морофоанализатор не умеет токенизировать сам (Mystem умеет, Pymorphy не умеет)

In [4]:
# два примера токенизации
from nltk.tokenize import word_tokenize
from gensim.utils import tokenize

In [ ]:
list(tokenize(text))

In [ ]:
word_tokenize(text)

## лемматизация и морфоанализ

### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно ему подавать сырой текст.

In [5]:
# сначала лемматизируем слова методом .lemmatize()
import string
print(text, "\n")

print(mystem.lemmatize(text))


зрелый фрукт на вкус очень сладок и обладает приятным сладковатым ароматом. в нём много витаминов и сахаров, но мало кислот. 

['зрелый', ' ', 'фрукт', ' ', 'на', ' ', 'вкус', ' ', 'очень', ' ', 'сладкий', ' ', 'и', ' ', 'обладать', ' ', 'приятный', ' ', 'сладковатый', ' ', 'аромат', '.', ' ', 'в', ' ', 'он', ' ', 'много', ' ', 'витамин', ' ', 'и', ' ', 'сахаров', ', ', 'но', ' ', 'мало', ' ', 'кислота', '.', '\n']


In [6]:
# метод .analyze() даст грамматическую информацию о словах

ms_analyzed = mystem.analyze(text)

print(ms_analyzed) # попробуйте разные индексы в этой переменной

метод .analyze() возвращает список словарей


каждый словарь имеет либо одно поле 'text' (когда попался пробел или пунктуация), либо analysis и text

* в analysis снова список словарей с вариантами разбора (первый самый вероятный)
* поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
* analysis - может быть пустым списком

In [7]:
# сделаем все красиво с индексами и доступом по ключам
# посмотрим на примере первого слова в тексте, поэтому индекс [0]


print('Слово из текста - ',ms_analyzed[0]['text'])
print('Разбор слова - ', ms_analyzed[0]['analysis'][0])
print('Лемма слова - ', ms_analyzed[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', ms_analyzed[0]['analysis'][0]['gr'])


Слово из текста -  зрелый
Разбор слова -  {'lex': 'зрелый', 'wt': 1, 'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)'}
Лемма слова -  зрелый
Грамматическая информация слова -  A=(вин,ед,полн,муж,неод|им,ед,полн,муж)


In [ ]:
# такое же,  но циклом, для всех слов в тексте

for idx,x in enumerate(ms_analyzed): # enumerate выдает элемент и его индекс

    if x.get('analysis'): # если есть словарь с морфо-разбором
        
        print('Слово из текста - ',ms_analyzed[idx]['text']) # получаем элементы по индексам
        print('Разбор слова - ', ms_analyzed[idx]['analysis'][0])
        print('Лемма слова - ', ms_analyzed[idx]['analysis'][0]['lex'])
        print('Грамматическая информация слова - ', ms_analyzed[idx]['analysis'][0]['gr'])
        print("\n")
    
    else: continue
    # если элемент без словаря с морфо-разбором, переходим к следующему элементу

In [8]:
# леммы можно достать в одну строчку

[elem['analysis'][0]['lex'] for elem in ms_analyzed if elem.get('analysis')]

['зрелый',
 'фрукт',
 'на',
 'вкус',
 'очень',
 'сладкий',
 'и',
 'обладать',
 'приятный',
 'сладковатый',
 'аромат',
 'в',
 'он',
 'много',
 'витамин',
 'и',
 'сахаров',
 'но',
 'мало',
 'кислота']

In [ ]:
# то же самое, что в предыдущей ячейке, но циклом

res = []

for elem in ms_analyzed:
    if elem.get('analysis'):
        res.append(elem['analysis'][0]['lex'])

res

#### дополнительные возможности Mystem

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/ и использовать в командной строке

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

### Pymorphy

Pymorphy - открытый и развивается ([можно поучаствовать на гитхабе](https://github.com/kmike/pymorphy2))


* [документация pymorphy](https://pythonhosted.org/pymorphy/)

У него нет встроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. Это лучше стемминга, но хуже майстема.

In [9]:
# основная функция - pymorphy.parse

pm_analyzed = [morph.parse(token) for token in list(tokenize(text))]

In [ ]:
pm_analyzed

In [ ]:
# пример с морфологической неоднозначностью

morph.parse("сахаров")

Она похожа на analyze в майстеме только возвращает список объектов Parse
* Первый в списке - самый вероятный разбор (у каждого есть score)
* Информация достается через атрибут (Parse.word)
* Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
части речи или другие категории

In [11]:
# сделаем красиво
print('Первое слово - ', pm_analyzed[0][0].word)
print('Лемма первого слова - ', pm_analyzed[0][0].normal_form)
print('Грамматическая информация первого слова - ', pm_analyzed[0][0].tag)
print('Часть речи первого слова - ', pm_analyzed[0][0].tag.POS)
print('Род первого слова - ', pm_analyzed[0][0].tag.gender)
print('Число первого слова - ',pm_analyzed[0][0].tag.number)
print('Падеж первого слова - ', pm_analyzed[0][0].tag.case)

Первое слово -  зрелый
Лемма первого слова -  зрелый
Грамматическая информация первого слова -  ADJF,Qual masc,sing,nomn
Часть речи первого слова -  ADJF
Род первого слова -  masc
Число первого слова -  sing
Падеж первого слова -  nomn


In [ ]:
# такое же,  но циклом, для всех слов в тексте

for idx,x in enumerate(pm_analyzed): # enumerate выдает элемент и его индекс

    print('Первое слово - ', pm_analyzed[idx][0].word)
    print('Лемма первого слова - ', pm_analyzed[idx][0].normal_form)
    print('Грамматическая информация первого слова - ', pm_analyzed[idx][0].tag)
    print('Часть речи первого слова - ', pm_analyzed[idx][0].tag.POS)
    print('Род первого слова - ', pm_analyzed[idx][0].tag.gender)
    print('Число первого слова - ',pm_analyzed[idx][0].tag.number)
    print('Падеж первого слова - ', pm_analyzed[idx][0].tag.case)
    print("\n")
    

### что можно дальше

Pymorphy и Mystem - не единственные морфоанализаторы для русского языка. Можно, например, посмотреть на [RNNmorph](https://github.com/IlyaGusev/rnnmorph) и [deeppavlov](http://docs.deeppavlov.ai/en/master/features/models/morphotagger.html).

А еще есть исследование, где сравнивали морфоанализаторы для русского  ([краткая версия](http://web-corpora.net/wsgi/mystemplus.wsgi/mystemplus/compare_table/), [статья](http://www.dialog-21.ru/media/3473/dereza.pdf))

 
И на последок, морфо-анализаторы для других яззыков:
- [UralicNLP](https://github.com/mikahama/uralicNLP)
- [hfst от Apertium](https://wiki.apertium.org/wiki/Hfst)
- [Stanza](https://stanfordnlp.github.io/stanza/)
- [SpaCy](https://spacy.io/usage/linguistic-features#morphology)
- [Trankit](https://trankit.readthedocs.io/en/latest/posdep.html)